```{contents}
```
## Tool Calling


**Tool calling** is a mechanism where an LLM can **decide to invoke external functions/tools** (APIs, databases, calculators, code, services) during reasoning, instead of answering purely from text generation.

> Tool calling turns an LLM from a text generator into an **action-taking system**.

The LLM:

1. Understands the user request
2. Decides **which tool** is needed
3. Produces a **structured tool call**
4. Receives the tool’s result
5. Uses it to produce the final answer

---

### Why Tool Calling Is Needed

LLMs alone cannot:

* Query live databases
* Fetch real-time data
* Perform exact calculations reliably
* Interact with systems (Jira, DBs, APIs)

Tool calling enables:

* Accuracy
* Fresh data
* Deterministic actions
* Enterprise integrations

---

### Where Tool Calling Fits in an LLM System

```
User Input
   ↓
LLM (Reasoning)
   ↓
Tool Selection
   ↓
Tool Execution
   ↓
Tool Result
   ↓
LLM (Final Answer)
```

This loop may run **once or multiple times**.

---

### Core Concepts in Tool Calling

#### Tool

A **function with a strict input schema**.

#### Tool Schema

Defines:

* Tool name
* Description
* Input parameters (JSON schema)

#### Tool Call

A **structured JSON output** generated by the LLM requesting tool execution.

#### Tool Result

The deterministic output returned by the tool.

---

### Tool Calling vs Prompt-Based APIs

| Aspect            | Prompt Only | Tool Calling |
| ----------------- | ----------- | ------------ |
| Determinism       | ❌           | ✅            |
| Live data         | ❌           | ✅            |
| Structured inputs | ❌           | ✅            |
| Enterprise-ready  | ❌           | ✅            |

---

### Tool Calling in LangChain

LangChain standardizes tool calling across model providers.

A tool is typically:

* A Python function
* Annotated with metadata
* Auto-converted to JSON schema

---

### Simple Tool Definition (LangChain)



In [26]:
from langchain_classic.tools import tool

@tool
def ticket_count(source: str) -> int:
    """Return total number of tickets for a given source"""
    return 128




LangChain automatically infers:

* Tool name: `ticket_count`
* Input schema: `{ "source": "string" }`
* Output type: `int`

---

### LLM with Tools (Chat Model)



In [27]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)




---

### Binding Tools to the LLM



In [28]:
llm_with_tools = llm.bind_tools([ticket_count])




This allows the model to **call the tool when needed**.

---

### Tool Calling Demonstration (End-to-End)

#### Prompt



In [29]:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You may use tools to answer questions."),
    ("human", "{input}")
])




---

### Chain



In [30]:
chain = prompt | llm_with_tools




---

### Invocation



In [31]:
response = chain.invoke(
    {"input": "Get ticket counts of Jira"}
)
print(response)


content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 60, 'total_tokens': 75, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_db64b059be', 'id': 'chatcmpl-CpIbaMa0FJWzUAfu50EUfEBxf9jH2', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b4234-50fc-73d2-be8a-a07e9594996a-0' tool_calls=[{'name': 'ticket_count', 'args': {'source': 'Jira'}, 'id': 'call_eVNTflzTGyX84QaCFAzwDoOS', 'type': 'tool_call'}] usage_metadata={'input_tokens': 60, 'output_tokens': 15, 'total_tokens': 75, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}




---

### What the LLM Produces (Conceptual)

```json
{
  "tool_calls": [
    {
      "name": "ticket_count",
      "arguments": {
        "source": "jira"
      }
    }
  ]
}
```

LangChain:

* Executes the tool
* Feeds the result back to the LLM
* Produces the final answer

---

### Final LLM Response

```text
There are 128 tickets in Jira.
```

---

### Tool Calling vs Agents

| Aspect         | Tool Calling | Agents     |
| -------------- | ------------ | ---------- |
| Decision       | Single-step  | Multi-step |
| Control        | High         | Lower      |
| Complexity     | Low          | High       |
| Predictability | High         | Medium     |

Tool calling is often used **inside agents**, but can be used standalone.

---

### Tool Calling vs MCP

| Aspect     | Tool Calling | MCP           |
| ---------- | ------------ | ------------- |
| Scope      | Single app   | Cross-app     |
| Transport  | In-process   | Client–server |
| Security   | Local        | Isolated      |
| Enterprise | Medium       | High          |

---

### Common Tool Types

#### Data Tools

* Database queries
* CSV readers
* Vector search

#### Action Tools

* Create ticket
* Update record
* Send email

#### Computation Tools

* Calculator
* Date math
* Validators

---

### Tool Calling Guardrails (Critical)

#### Strict Schemas

* Prevent malformed calls

#### Deterministic Tools

* No side effects without validation

#### Allowlist Tools

* Only approved tools exposed

#### Low Temperature

* Reduces tool misuse

---

### Failure Modes

#### Hallucinated Tool Calls

* Prevented by schema validation

#### Wrong Tool Selection

* Mitigated with better descriptions

#### Infinite Tool Loops

* Prevented with max-steps limits

---

### Production Best Practices

* Use clear tool descriptions
* Keep tool interfaces minimal
* Validate tool inputs
* Log all tool calls
* Separate read vs write tools
* Add human approval for destructive actions

---

### When to Use Tool Calling

* Real-time data access
* System integrations
* Deterministic workflows
* Enterprise automation

---

### When NOT to Use Tool Calling

* Pure text generation
* Creative writing
* Static knowledge answers

---

### Interview-Ready Summary

> “Tool calling allows an LLM to invoke external functions using structured schemas, enabling deterministic actions, real-time data access, and system integrations. In LangChain, tools are first-class abstractions that power production-grade agentic workflows.”

---

### Rule of Thumb

* **Knowledge → RAG**
* **Actions → Tools**
* **Complex decisions → Agents**
* **Enterprise integration → Tool calling + guardrails**